<a href="https://colab.research.google.com/github/Puass/LinearRegression/blob/main/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data


In [213]:
df=pd.read_csv("Melbourne_housing_FULL.csv")

In [214]:
df

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.81140,145.01160,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0


#EDA and Preprocess


In [215]:
def print_missing_values(df):
  column_missing_values = df.isnull().sum().sort_values(ascending=False)
  missing_df = pd.DataFrame(column_missing_values, columns=['Number of missing values'])

  print(column_missing_values)

In [216]:
print_missing_values(df)

BuildingArea     21115
YearBuilt        19306
Landsize         11810
Car               8728
Bathroom          8226
Bedroom2          8217
Longtitude        7976
Lattitude         7976
Price             7610
Regionname           3
CouncilArea          3
Propertycount        3
Postcode             1
Distance             1
Address              0
Date                 0
SellerG              0
Method               0
Type                 0
Rooms                0
Suburb               0
dtype: int64


In [217]:
df.dropna(subset=['Longtitude', 'Lattitude','Regionname', 'CouncilArea', 'Propertycount', 'Postcode', 'Distance'], inplace=True)

In [218]:
#Fill Car Column
mode_car = df['Car'].mode()[0]
df['Car'].fillna(mode_car, inplace=True)

#Fill Bathroom
df['Bathroom'].fillna(df.groupby('Rooms')['Bathroom'].transform(lambda x: x.mode()[0]), inplace=True)

#Fill Bedroom2
df['Bedroom2'].fillna(df.groupby('Rooms')['Bedroom2'].transform(lambda x: x.mode()[0]), inplace=True)

columns_to_impute = [ 'YearBuilt', 'Landsize', 'Price']

for column in columns_to_impute:
    median_value = df[column].median()
    df[column].fillna(median_value, inplace=True)

print_missing_values(df)


df.drop_duplicates(inplace=True)

BuildingArea     13148
Suburb               0
Bathroom             0
Regionname           0
Longtitude           0
Lattitude            0
CouncilArea          0
YearBuilt            0
Landsize             0
Car                  0
Bedroom2             0
Address              0
Postcode             0
Distance             0
Date                 0
SellerG              0
Method               0
Price                0
Type                 0
Rooms                0
Propertycount        0
dtype: int64


In [219]:
#df['TP']=(pd.to_datetime("today")-pd.to_datetime(df['Date'])).dt.days

In [220]:
df.select_dtypes(include='object').columns

Index(['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea',
       'Regionname'],
      dtype='object')

In [221]:
df.select_dtypes(include='object').columns
df.drop(columns=['Date', 'Suburb','Address','BuildingArea'], inplace=True)
obj_cols = df.select_dtypes(include='object').columns

from scipy import stats

numerical_columns = [col for col in df.columns if col not in obj_cols]

print_missing_values(df)


# Aykırı değerleri filtrelemek için bir işlev tanımlayın
def filter_outliers(series):
    z_scores = np.abs(stats.zscore(series))
    threshold = 3  # Aykırı değer olarak kabul edilen z-score eşik değeri
    return series[(z_scores < threshold)]

# Aykırı değerleri filtreleyin ve yeni bir DataFrame oluşturun
filtered_df = df.copy()
for col in numerical_columns:
    filtered_df[col] = filter_outliers(df[col])

# Filtrelenmiş DataFrame'i kullanabilirsiniz
print(filtered_df)

print_missing_values(df)

from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()
df[obj_cols] = oe.fit_transform(df[obj_cols])

df.to_csv('preprocessed_data.csv', index=False)

Rooms            0
Car              0
Regionname       0
Longtitude       0
Lattitude        0
CouncilArea      0
YearBuilt        0
Landsize         0
Bathroom         0
Type             0
Bedroom2         0
Postcode         0
Distance         0
SellerG          0
Method           0
Price            0
Propertycount    0
dtype: int64
       Rooms Type      Price Method  SellerG  Distance  Postcode  Bedroom2  \
0        2.0    h   910000.0     SS   Jellis       2.5    3067.0       2.0   
1        2.0    h  1480000.0      S   Biggin       2.5    3067.0       2.0   
2        2.0    h  1035000.0      S   Biggin       2.5    3067.0       2.0   
3        3.0    u   910000.0     VB   Rounds       2.5    3067.0       3.0   
4        3.0    h  1465000.0     SP   Biggin       2.5    3067.0       3.0   
...      ...  ...        ...    ...      ...       ...       ...       ...   
34851    3.0    h  1101000.0      S      Jas       6.3    3013.0       3.0   
34852    4.0    h  1480000.0     PI     

In [222]:
df

,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,2,0.0,910000.0,6.0,142.0,2.5,3067.0,2.0,1.0,1.0,126.0,1970.0,31.0,-37.80140,144.99580,2.0,4019.0
1,2,0.0,1480000.0,2.0,28.0,2.5,3067.0,2.0,1.0,1.0,202.0,1970.0,31.0,-37.79960,144.99840,2.0,4019.0
2,2,0.0,1035000.0,2.0,28.0,2.5,3067.0,2.0,1.0,0.0,156.0,1900.0,31.0,-37.80790,144.99340,2.0,4019.0
3,3,2.0,910000.0,7.0,279.0,2.5,3067.0,3.0,2.0,1.0,0.0,1970.0,31.0,-37.81140,145.01160,2.0,4019.0
4,3,0.0,1465000.0,5.0,28.0,2.5,3067.0,3.0,2.0,0.0,134.0,1900.0,31.0,-37.80930,144.99440,2.0,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34851,3,0.0,1101000.0,2.0,139.0,6.3,3013.0,3.0,1.0,2.0,288.0,1970.0,16.0,-37.81095,144.88516,6.0,6543.0
34852,4,0.0,1480000.0,0.0,139.0,6.3,3013.0,4.0,1.0,3.0,593.0,1970.0,16.0,-37.81053,144.88467,6.0,6543.0
34853,2,0.0,888000.0,5.0,302.0,6.3,3013.0,2.0,2.0,1.0,98.0,2018.0,16.0,-37.81551,144.88826,6.0,6543.0
34854,2,1.0,705000.0,2.0,139.0,6.3,3013.0,2.0,1.0,2.0,220.0,2000.0,16.0,-37.82286,144.87856,6.0,6543.0


# Models

In [223]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max normalizasyonu yapmak için bir MinMaxScaler nesnesi oluşturalım
scaler = MinMaxScaler()

# DataFrame içindeki sayısal sütunları normalleştirin
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Normalleştirilmiş DataFrame'i görüntüleyin
print(df_normalized)

          Rooms  Type     Price  Method   SellerG  Distance  Postcode  \
0      0.066667   0.0  0.074224   0.750  0.394444  0.051975  0.068507   
1      0.066667   0.0  0.125506   0.250  0.077778  0.051975  0.068507   
2      0.066667   0.0  0.085470   0.250  0.077778  0.051975  0.068507   
3      0.133333   1.0  0.074224   0.875  0.775000  0.051975  0.068507   
4      0.133333   0.0  0.124157   0.625  0.077778  0.051975  0.068507   
...         ...   ...       ...     ...       ...       ...       ...   
26875  0.133333   0.0  0.091408   0.250  0.386111  0.130977  0.013292   
26876  0.200000   0.0  0.125506   0.000  0.386111  0.130977  0.013292   
26877  0.066667   0.0  0.072245   0.625  0.838889  0.130977  0.013292   
26878  0.066667   0.5  0.055780   0.250  0.386111  0.130977  0.013292   
26879  0.066667   0.0  0.084121   0.000  0.719444  0.130977  0.013292   

       Bedroom2  Bathroom       Car  Landsize  YearBuilt  CouncilArea  \
0      0.066667  0.083333  0.038462  0.000291   0.

In [224]:
from sklearn.model_selection import train_test_split


X = df.drop(['Price'],axis=1) # Features
y=df['Price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)





In [225]:
print_missing_values(X)

Rooms            0
Type             0
Method           0
SellerG          0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
YearBuilt        0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64


In [226]:
from sklearn.preprocessing import RobustScaler

ro_scaler = RobustScaler()
X_train = ro_scaler.fit_transform(X_train)
X_test = ro_scaler.fit_transform(X_test)

In [227]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import randint, uniform
import numpy as np

from sklearn.model_selection import GridSearchCV

models = {
    'Lasso': {
        'model': Lasso(),
        'param_grid': {
            'alpha': [0.01, 0.1, 1.0],
            'fit_intercept': [True, False],
            'precompute': [True, False],
            'positive': [True, False],
            'selection': ['cyclic', 'random']
        }
    },
    'LinearRegression': {
        'model': LinearRegression(),
        'param_grid': {
            'fit_intercept': [True, False]
        }
    },
    'Ridge': {
        'model': Ridge(),
        'param_grid': {
            'alpha': [0.01, 0.1, 1.0],
            'fit_intercept': [True, False],
            'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
        }
    },
    'ElasticNet': {
        'model': ElasticNet(),
        'param_grid': {
            'alpha': [0.01, 0.1, 1.0],
            'l1_ratio': [0.1, 0.5, 0.9],
            'fit_intercept': [True, False],
            'positive': [True, False],
            'selection': ['cyclic', 'random']
        }
    },
    'KNeighborsRegressor': {
        'model': KNeighborsRegressor(),
        'param_grid': {
            'n_neighbors': [5, 10, 15, 20, 25],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'leaf_size': [20, 30, 40],
        }
    },
    'RandomForestRegressor': {
        'model': RandomForestRegressor(),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 5, 10],
            'bootstrap': [True, False],
        }
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.5],
            'subsample': [0.7, 0.8, 0.9, 1.0],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 5, 10],
            'max_features': ['auto', 'sqrt', 'log2'],
        }
    },
    'AdaBoostRegressor': {
        'model': AdaBoostRegressor(),
        'param_grid': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.5],
            'loss': ['linear', 'square', 'exponential'],
        }
    }
}

# Modelleri dönerken Grid Search ile parametre optimizasyonu yapın
for model_name, model_info in models.items():
    model = model_info['model']
    param_grid = model_info['param_grid']

    grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    print(f"{model_name}:")
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")
    print("\n")


Lasso:
Best Parameters: {'alpha': 1.0, 'fit_intercept': True, 'positive': False, 'precompute': True, 'selection': 'random'}
Mean Squared Error: 219119664896.93445


LinearRegression:
Best Parameters: {'fit_intercept': True}
Mean Squared Error: 219121220953.84653


Ridge:
Best Parameters: {'alpha': 0.01, 'fit_intercept': True, 'solver': 'lsqr'}
Mean Squared Error: 218908509198.06473


ElasticNet:
Best Parameters: {'alpha': 0.01, 'fit_intercept': True, 'l1_ratio': 0.9, 'positive': False, 'selection': 'random'}
Mean Squared Error: 217628453165.39468


KNeighborsRegressor:
Best Parameters: {'algorithm': 'ball_tree', 'leaf_size': 30, 'n_neighbors': 15, 'weights': 'distance'}
Mean Squared Error: 177217466906.28577




/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: ignored